# Exploring Chinese Lexicon-based Tokenizer

- Top F1=0.83 regardless of delimiting symbols added to the lexicon (higher than Freedom-peak-based top F1=0.71)
- Top F1 based on length-driven parsing, not weight-driven or "hybrid" (legth-times-logweight-driven)
- Most of Errors (0.17) are due to "unknown" words but typically have no impaxt on the meaning  
- Not adding delimiters to lexicon does not have an effect
- Adding threshold on word freqency may improve F1 from 0.82 to 0.83 on one corpus (SUBTLEX-CH-WF) with very tiny threshold of 0.000001
- Precision of word discovery of Freedom-peak-based tokenizer is at least 0.92 (after correction for major out-of-refernce-lexicon words), comparable with one based on Jieba tokenizer (0.94)



In [41]:
import os, sys, gc
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [2]:
path = '../../nlp/corpora/Chinese/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['zh'])
print(len(test_texts))
test_df[['zh']]

100


,zh
0,然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
1,当他们买了保险的，按照保险合同的话，是要赔三十万的
2,需要意识到买了一个保险的重要性
3,其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
4,这代父母真的很有必要去买一个保险
...,...
95,蚂蚁保险不止有车险，我看到上面也有各种。
96,但是买房子呢，除了一笔首款付下去之后，每个月的贷款相当于租金
97,这种的投资性的理财，对于个人的财产来说，是具有一定风险
98,如果说你的投资方向正确，那么它是远比存款这一方面要具有更大的优势


In [3]:
for text in test_texts:
    print(text)

然后医疗保险呢？就是我们家，不论是大人啊还是小孩都会去买一个保险
当他们买了保险的，按照保险合同的话，是要赔三十万的
需要意识到买了一个保险的重要性
其实这种现象是真的很普遍，因为比如说你买一个人身意外险你那个你买的越多你那个保额就越多
这代父母真的很有必要去买一个保险
嗯，对现在就是说很提倡买保险
车一定要全款买，房子可以贷款买
可以去买保险，保险当然分为很多类
医疗保险还是很重要的
就是保险公司来出这一部分钱
相互宝，我不知道你有没有了解过，它是支付宝里面的一种保险
买房子其实也就是一个投资
你有没有去了解过平安保险公司的培训啊
如果要是存银行的话，利息利滚利是多少
这个保险在西方国家已经很成熟了
保险行业很多啊，你看它覆盖的面积特别广
但是，其实也是在鼓励你买商业保险和人寿保险
卖保险的都是靠的人脉
呃，你买的社保也是保险呀，然后你的车险也是保险呀
但是银行的钱，你如果是拿出来的话，属于是一种贷款
现在好像转向那个负利率
借银行的钱，我真的没试过
他是没有利息的，所以他就把它放出去，以后再收回来
如果是你征信不好的话，他也不会给钱你用
是他是贷款买的还是全款
嗯，他们就是就是有时候办一张银行卡的时候，程序特别多
各种银行，还有五大银行
国债券也算是证券的一种吗
其实做股票的话嗯相它是算是风险系数比较高的
嗯收益不会太高就跟银行利率是差不多的
然后就用来炒房用啊，等它升值了之后卖一套房，然后还清贷款，可能自己还赚了那个几十万
入投资入门的话，第一步就是说你要掌握基础知识
汇率的标价方式还有他的直接标价法和间接标价法
它始终是一种投资的行为，投资的模式，投资的方式
如果有闲钱呢买一套公寓来投资呢？
变成资金，投资其他的方面
有了贷款就有了压力，也就有了挣钱的动力，这个我也认同
方不方便开通网上银行啊
那你上次投资的钱从哪来的
银行的政策的改变让他们不能贷款了
保险公司呢它也有理财产品的
好像我们那个保险公司是两点八的利息保底
所以很多人对保险很反感
他们宁愿把那个钱去存银行吃利息
提前五年理财真好，因为理财本身就是一个很轻松的事情
就是你把你的生活费支出的一小部分作一个储蓄
债券基金里这种基金的话，它是收益是比较稳定的，就跟存定期一样，但是比定期的钱要多
它理论上可以开多少张这种承兑汇票
就是说我们每个人都应该有自己的理财方式，不管你是你理什么财不管你有多少钱
是

In [26]:
# https://github.com/fxsjy/jieba
zh_jieba_tokenizer = JiebaTokenizer()

# http://www.chineselexicaldatabase.com/download.php - used below
zh_lex_tokenizer = LexiconIndexedTokenizer(lexicon = list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word']))
zh_lex_delimited_tokenizer = LexiconIndexedTokenizer(lexicon = list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word'])+list(delimiters))
print(zh_lex_tokenizer.count_params())
print(zh_lex_delimited_tokenizer.count_params())

# https://www.plecoforums.com/threads/word-frequency-list-based-on-a-15-billion-character-corpus-bcc-blcu-chinese-corpus.5859/
bcc_lex = list(pd.read_csv(os.path.join(path,'lexicon/bcc_global_wordfreq.release_UTF-8.txt'),sep='\t').to_records(index=False))
top_weight = max([i[1] for i in bcc_lex],key=lambda item:item)
print(top_weight)
bcc_lex_delimited = bcc_lex + [(i, top_weight) for i in list(delimiters)]
zh_bcc0_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=0)
zh_bcc1_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=1)
zh_bcc2_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex,sortmode=2)
zh_bcc0_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex_delimited,sortmode=0)
zh_bcc1_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex_delimited,sortmode=1)
zh_bcc2_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=bcc_lex_delimited,sortmode=2)
print(zh_bcc0_tokenizer.count_params())

# http://crr.ugent.be/programs-data/subtitle-frequencies/subtlex-ch
sub_lex = list(pd.read_csv(os.path.join(path,'lexicon/SUBTLEX-CH-WF.txt'),sep='\t')[['Word','WCount']].to_records(index=False))
top_weight = max([i[1] for i in sub_lex],key=lambda item:item)
print(top_weight)
sub_lex_delimited = sub_lex + [(i, top_weight) for i in list(delimiters)]
zh_sub0_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=0)
zh_sub1_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=1)
zh_sub2_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex,sortmode=2)
zh_sub0_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex_delimited,sortmode=0)
zh_sub1_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex_delimited,sortmode=1)
zh_sub2_delimited_tokenizer = LexiconIndexedTokenizer(lexicon=sub_lex_delimited,sortmode=2)
print(zh_sub0_tokenizer.count_params())  


48644
48678
943370349
1036991
1682530
99121


In [29]:
full_lex = [(i,1.0) for i in list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word'])+list(delimiters)] + bcc_lex + sub_lex
zh_full_tokenizer = LexiconIndexedTokenizer(lexicon = full_lex)
print(zh_full_tokenizer.count_params())  


1075763


In [30]:
#http://www.chineselexicaldatabase.com/download.php
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_lex_tokenizer,debug=False))
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_lex_delimited_tokenizer,debug=False))
print()
#https://www.plecoforums.com/threads/word-frequency-list-based-on-a-15-billion-character-corpus-bcc-blcu-chinese-corpus.5859/
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc0_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc1_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc2_tokenizer,debug=False))#sort by len and freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc0_delimited_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc1_delimited_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc2_delimited_tokenizer,debug=False))#sort by len and freq
print()
#http://crr.ugent.be/programs-data/subtitle-frequencies/subtlex-ch
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub0_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub1_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub2_tokenizer,debug=False))#sort by len and freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub0_delimited_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub1_delimited_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub2_delimited_tokenizer,debug=False))#sort by len and freq
print()
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_full_tokenizer,debug=False))


0.82
0.82

0.79
0.48
0.79
0.79
0.48
0.79

0.82
0.49
0.78
0.82
0.49
0.78

0.83


In [20]:
print(len(bcc_lex))
filter_thresholds = [0.000001,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(bcc_lex,t)
    print(t,len(lex))
    zh_bcc0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0)
    zh_bcc1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1)
    zh_bcc2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2)
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_bcc2_tokenizer,debug=False))#sort by len and freq
    print()


1048575
1e-06 141441
0.79
0.48
0.79

1e-05 52857
0.79
0.48
0.79

0.0001 13983
0.76
0.48
0.76

0.001 2499
0.66
0.47
0.66

0.01 185
0.5
0.44
0.5



In [21]:
print(len(sub_lex))
filter_thresholds = [0.000001,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(sub_lex,t)
    print(t,len(lex))
    zh_sub0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0)
    zh_sub1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1)
    zh_sub2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2)
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_sub2_tokenizer,debug=False))#sort by len and freq
    print()

99121
1e-06 69005
0.83
0.49
0.78

1e-05 29174
0.82
0.49
0.77

0.0001 8343
0.74
0.49
0.72

0.001 1621
0.63
0.49
0.63

0.01 233
0.55
0.46
0.55



In [23]:
#zh_lex_tokenizer = LexiconIndexedTokenizer(lexicon = list(pd.read_csv(os.path.join(path,'lexicon/chineselexicaldatabase2.1.txt'))['Word']))
print(evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,zh_lex_tokenizer,debug=False))
print()
for text in test_texts:
    expected = zh_jieba_tokenizer.tokenize(text)
    actual = zh_lex_tokenizer.tokenize(text)
    f1 = calc_f1(expected,actual)
    if f1 < 1:
        print(expected)
        print(actual)
        print(round(f1,2))


0.82

['然后', '医疗保险', '呢', '？', '就是', '我们', '家', '，', '不论是', '大人', '啊', '还是', '小孩', '都', '会', '去', '买', '一个', '保险']
['然后', '医疗', '保险', '呢', '？', '就是', '我们', '家', '，', '不论是', '大人', '啊', '还是', '小孩', '都会', '去', '买', '一个', '保险']
0.84
['当', '他们', '买', '了', '保险', '的', '，', '按照', '保险合同', '的话', '，', '是', '要', '赔', '三十万', '的']
['当', '他们', '买', '了', '保险', '的', '，', '按照', '保险', '合同', '的话', '，', '是', '要', '赔', '三', '十', '万', '的']
0.8
['其实', '这种', '现象', '是', '真的', '很', '普遍', '，', '因为', '比如说', '你', '买', '一个', '人身', '意外险', '你', '那个', '你', '买', '的', '越', '多', '你', '那个', '保额', '就', '越', '多']
['其实', '这', '种', '现象', '是', '真的', '很', '普遍', '，', '因为', '比如说', '你', '买', '一个', '人身', '意外', '险', '你', '那个', '你', '买', '的', '越', '多', '你', '那个', '保', '额', '就', '越', '多']
0.85
['这代', '父母', '真的', '很', '有', '必要', '去', '买', '一个', '保险']
['这', '代', '父母', '真的', '很', '有', '必要', '去', '买', '一个', '保险']
0.86
['车', '一定', '要', '全款', '买', '，', '房子', '可以', '贷款', '买']
['车', '一定', '要', '全', '款', '买', '，', '房子', '可以', '贷款', '买']
0.86
['

## Explore validity of the discovered lexicon

- Precision of word discovery of Freedom-peak-based tokenizer is at least 0.92 (after correction for major out-of-refernce-lexicon words), comparable with one based on Jieba tokenizer (0.94)


In [ ]:
# use SOTA
base = FreedomTokenizer(name='data/models/zh_train_chars_2a',max_n=2,mode='chars',debug=False)
model_compress_with_loss(base.model,0.001)
test_tokenizer = FreedomBasedTokenizer(base,'peak-','peak+')
test_tokenizer.set_options(nlist = [2], threshold=0.01) # expected F1=0.71


In [50]:
expected = {}
actual = {}
f1 = evaluate_tokenizer_f1(test_texts,zh_jieba_tokenizer,test_tokenizer,expected_collector=expected,actual_collector=actual)
print(f1)

0.71


In [80]:
#collected tokens

print('total relevant false precision')

expected_count = sum([expected[key] for key in expected])
relevant_count = sum([expected[key] for key in expected if key in zh_full_tokenizer.fulldict])
irrelevant_count = sum([expected[key] for key in expected if not key in zh_full_tokenizer.fulldict])
print(expected_count,relevant_count,irrelevant_count,relevant_count/expected_count,(relevant_count+86+8+2)/expected_count)

actual_count = sum([actual[key] for key in actual])
relevant_count = sum([actual[key] for key in actual if key in zh_full_tokenizer.fulldict])
irrelevant_count = sum([actual[key] for key in actual if not key in zh_full_tokenizer.fulldict])
print(actual_count,relevant_count,irrelevant_count,relevant_count/actual_count,(relevant_count+84+7+2)/actual_count)


total relevant false precision
1268 1096 172 0.8643533123028391 0.9400630914826499
1340 1142 198 0.8522388059701492 0.9216417910447762


In [73]:
#jieba tokenizer
misses = sorted([(key,expected[key]) for key in expected if not key in zh_full_tokenizer.fulldict],key = lambda x: x[1],reverse=True)
misses[:10]

[('，', 86),
 ('一种', 8),
 ('商业保险', 5),
 ('医疗保险', 3),
 ('？', 3),
 ('保险合同', 3),
 ('全款', 2),
 ('有没有', 2),
 ('买房子', 2),
 ('车险', 2)]

In [74]:
#freedom tokenizer
misses = sorted([(key,actual[key]) for key in actual if not key in zh_full_tokenizer.fulldict],key = lambda x: x[1],reverse=True)
misses[:10]

[('，', 84),
 ('一种', 7),
 ('呢？', 3),
 ('说你', 2),
 ('你买', 2),
 ('全款', 2),
 ('这一', 2),
 ('车险', 2),
 ('等它', 2),
 ('一套', 2)]

In [81]:
print('，' in zh_full_tokenizer.fulldict) # out of reference lexicon
print('一种' in zh_full_tokenizer.fulldict) # "A sort of" - out of reference lexicon
print('全款' in zh_full_tokenizer.fulldict) # "Full payment" - out of reference lexicon


False
False
False


In [82]:
#cleanup
del base
del test_tokenizer
gc.collect()


104